In [ ]:
from dotenv import load_dotenv, find_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions

load_dotenv(find_dotenv())

In [ ]:
model = OllamaFunctions(model="phi3:latest", format="json", keep_alive=-1)

In [ ]:
model.invoke('hi there!, tell why 2+2 is 5?')

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import random
from langchain_core.tools import tool
from langchain_core.tools import tool
import random
from langchain_core.utils.function_calling import convert_to_openai_function


@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city. The API is available randomly, approximately every third call."""

    if random.randint(1, 3) == 1:  # Approximately one-third chance
        return "Sunny, 22°C"
    else:
        return "Service temporarily unavailable"


tools = [fake_weather_api]
functions = [convert_to_openai_function(tool) for tool in tools]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(model, functions, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """
    Get the weather for a location

    """
    return "weather in " + location + " is sunny with a high of 75 degrees"

@tool
def get_population(location: str) -> str:
    """
    Get the population of a location

    """
    return "the population of " + location + " is 100,000"


tools = [get_weather, get_population]
functions = [convert_to_openai_function(t) for t in tools]
tool_mapping = {"get_weather": get_weather, "get_population": get_population}

In [ ]:
functions

In [ ]:
model = model.bind_tools(functions)

In [ ]:
response = model.invoke("what is the weather in Boston and its population?")
response

In [ ]:
response.additional_kwargs["function_call"]

In [ ]:
response.additional_kwargs | itemgetter("arguments")

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent